In [ ]:
#RDDMS demo via Rest-APIs 
#documented in the SWAGGER pages
#http://10.0.1.4:8003/Reservoir/v2#/Resources/

In [ ]:
#Libs needed
!pip install requests
!pip install pandas
!pip install --upgrade matplotlib

In [ ]:
import requests
import json
import urllib.parse
import pandas as pd
import numpy as np

In [ ]:
# Copy your az bearer token to variable bearer_key: PS F:\> az account get-access-token 

bearer_key = ''

RESERVOIR_DDMS_HOST = 'http://osdubootcamp.azure-api.net/reservoir/v2'

#headers = {
#    'accept': 'application/json',
#}
#response = requests.get(RESERVOIR_DDMS_HOST+'/auth/token', headers=headers)
#response.json()['token']


In [ ]:
#List the available data space(s)

In [ ]:
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {bearer_key}'
}

params = {
    '$skip': '0',
    '$top': '30',
}

response = requests.get(RESERVOIR_DDMS_HOST+'/dataspaces', params=params, headers=headers)

In [ ]:
print(*response, sep='\n')
print('-----------------')

SelectedDataspace = 0

#You need to URL encode the namespace
print('Dataspace name:', response.json()[SelectedDataspace]['path'])
dataspace_name = urllib.parse.quote(response.json()[SelectedDataspace]['path'], safe="")
print('URL encoded namespace:',dataspace_name)

In [ ]:
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {bearer_key}'
}

response = requests.get(f'{RESERVOIR_DDMS_HOST}/dataspaces/{dataspace_name}/resources', headers=headers)

In [ ]:
#print(json.dumps(response.json(),indent=4))
response_flat = pd.json_normalize(response.json()).to_dict(orient='records')
response_df = pd.DataFrame.from_dict(response_flat)
print(response_df)

In [ ]:
# List all object given an object type

In [ ]:
response = requests.get(
    f'{RESERVOIR_DDMS_HOST}/dataspaces/{dataspace_name}/resources/resqml20.obj_Grid2dRepresentation',
    params=params,
    headers=headers,
)

In [ ]:
print(json.dumps(response.json(),indent=4))

In [ ]:
SelectedSurface =  0 # fisrt and only object in dataspace
uuid = urllib.parse.quote(response.json()[SelectedSurface]['uri'].split('(')[-1].replace(')','')) 

print('uuid:', uuid)

In [ ]:
print(json.dumps(response.json(),indent=4))
response_flat = pd.json_normalize(response.json()).to_dict(orient='records')
response_df = pd.DataFrame.from_dict(response_flat)
print(response_df)

In [ ]:
headers = {
    'accept': 'application/x-resqml+xml',
    'Authorization': f'Bearer {bearer_key}'
}

params = {
    '$format': 'json',
    'arrayMetadata': 'false',
    'arrayValues': 'false',
    'referencedContent': 'true',
}

response = requests.get(
    f'{RESERVOIR_DDMS_HOST}/dataspaces/{dataspace_name}/resources/resqml20.obj_Grid2dRepresentation/{uuid}',
    params=params,
    headers=headers,
)
print(json.dumps(response.json(),indent=4))

In [ ]:
headers = {
    'accept': 'application/x-resqml+xml',
    'Authorization': f'Bearer {bearer_key}'
}

params = {
    '$format': 'json',
    'arrayMetadata': 'false',
    'arrayValues': 'false',
    'referencedContent': 'true',
}


response = requests.get(
    f'{RESERVOIR_DDMS_HOST}/dataspaces/{dataspace_name}/resources/resqml20.obj_Grid2dRepresentation/{uuid}/arrays',
    params=params,
    headers=headers,
)
print(json.dumps(response.json(),indent=4))

In [ ]:
uuid_url = urllib.parse.quote(response.json()[0]['uid']['pathInResource'], safe="")
print('uuid_url:', uuid_url)

In [ ]:
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {bearer_key}'
    }

params = {
    'format': 'json'
}

response = requests.get(
    f'{RESERVOIR_DDMS_HOST}/dataspaces/{dataspace_name}/resources/resqml20.obj_Grid2dRepresentation/{uuid}/arrays/{uuid_url}',
    params=params,
    headers=headers,
)

In [ ]:
print(response.text)

In [ ]:
print('Data length:',len(response.json()['data']['data']))
print('Dimensions:',response.json()['data']['dimensions'])
assert len(response.json()['data']['data']) == ( int(response.json()['data']['dimensions'][0]) * int(response.json()['data']['dimensions'][1]) )

In [ ]:
xy_array = np.reshape(response.json()['data']['data'], (int(response.json()['data']['dimensions'][0]), int(response.json()['data']['dimensions'][1]))).T

In [ ]:
len(xy_array[:,:])

In [ ]:
z = np.array(response.json()['data']['data'], dtype=np.float32)
z_array = np.reshape(z, (int(response.json()['data']['dimensions'][0]), int(response.json()['data']['dimensions'][1])))
print(xy_array.shape)

In [ ]:
#https://matplotlib.org/stable/gallery/mplot3d/custom_shaded_3d_surface.html#sphx-glr-gallery-mplot3d-custom-shaded-3d-surface-py
from matplotlib import cbook
from matplotlib import cm
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
import numpy as np

z = z_array
nrows, ncols = z.shape
print(nrows, ncols)
x = np.linspace(0, 5, ncols)
y = np.linspace(0, 5, nrows)
x, y = np.meshgrid(x, y)

fig, ax = plt.subplots(subplot_kw=dict(projection='3d'))
surf = ax.plot_surface(x, y, z)

plt.show()